In [1]:
from tensorscaling import scale, unit_tensor, marginal
import numpy as np

# Tensor scaling

Scale 3x3x3 unit tensor to certain non-uniform marginals:

In [2]:
shape = [3, 3, 3]
targets = [(0.5, 0.25, 0.25), (0.4, 0.3, 0.3), (0.7, 0.2, 0.1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-4)
res

Result(success=True, iterations=92, max_dist=7.441191287479101e-05, ..., log_cap=-0.22911478596326992)

We can also access the scaling matrices and the final scaled state:

In [3]:
print(res.gs[0], "\n")
print(res.gs[1], "\n")
print(res.gs[2])

[[ 1.45035287+0.j          0.        +0.j          0.        +0.j        ]
 [-0.14729358-0.17484196j  0.62605858+0.j          0.        +0.j        ]
 [ 0.22074165+0.05528096j  0.14299424+0.02410229j  0.89191091+0.j        ]] 

[[ 0.91757562+0.j          0.        +0.j          0.        +0.j        ]
 [ 0.44882921+0.84146195j  1.28560685+0.j          0.        +0.j        ]
 [-0.02146844+0.62702483j  0.39294764+0.63233859j  0.97247639+0.j        ]] 

[[ 1.5741347 +0.j          0.        +0.j          0.        +0.j        ]
 [ 0.06049845-0.15828112j  0.61215079+0.j          0.        +0.j        ]
 [ 0.0567228 +0.01677289j -0.008868  -0.05141692j  0.53184379+0.j        ]]


Let's now check that the W tensor *cannot* be scaled to uniform marginals:

In [4]:
shape = [2, 2, 2, 2]
W = np.zeros(shape)
W[1, 0, 0, 0] = W[0, 1, 0, 0] = W[0, 0, 1, 0] = W[0, 0, 0, 1] = 0.5
targets = [(0.5, 0.5)] * 4

scale(W, targets, eps=1e-4, max_iterations=100)

Result(success=False, iterations=100, max_dist=0.5934648479435459, ..., log_cap=None)

To see more clearly what is going on, we can set the `verbose` flag:

In [5]:
res = scale(W, targets, eps=1e-4, max_iterations=10, verbose=True)

scaling tensor of shape (2, 2, 2, 2) and type float64
target spectra:
  0: (np.float64(0.5), np.float64(0.5))
  1: (np.float64(0.5), np.float64(0.5))
  2: (np.float64(0.5), np.float64(0.5))
  3: (np.float64(0.5), np.float64(0.5))
#000: max_dist = 0.35355339 @ sys = 3
#001: max_dist = 0.47140452 @ sys = 2
#002: max_dist = 0.56568542 @ sys = 0
#003: max_dist = 0.62853936 @ sys = 1
#004: max_dist = 0.58232323 @ sys = 3
#005: max_dist = 0.59305730 @ sys = 2
#006: max_dist = 0.59545834 @ sys = 0
#007: max_dist = 0.59262283 @ sys = 1
#008: max_dist = 0.59353004 @ sys = 3
#009: max_dist = 0.59357133 @ sys = 2
#010: max_dist = 0.59340661 @ sys = 0
did not converge!


We see that at each point in the algorithm, one of the marginals has Frobenius distance $\approx 0.59$ to being uniform. Indeed, we know that the entanglement polytope of the W tensor does not include the point corresponding to uniform marginals -- see [here](https://www.entanglement-polytopes.org/four_qubits) for an interactive visualization!

# Tuples of matrices and tensors

We can just as well only prescribe the desired spectra for subsystems.
Note that prescribing two out of three marginals amounts to *operator scaling*.

In [6]:
shape = [3, 3, 3]
targets = [(0.4, 0.3, 0.3), (0.7, 0.2, 0.1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-6)
res

Result(success=True, iterations=55, max_dist=8.278433372817129e-07, ..., log_cap=-0.37573462134826935)

Indeed, the last two marginals are as prescribed, while the first marginal is arbitrary.

In [7]:
print(marginal(res.psi, 0).round(5), "\n")
print(marginal(res.psi, 1).round(5), "\n")
print(marginal(res.psi, 2).round(5))

[[ 3.0003e-01+0.j       1.7000e-04-0.00059j -4.7000e-04+0.00021j]
 [ 1.7000e-04+0.00059j  3.5636e-01+0.j      -2.7570e-02-0.02119j]
 [-4.7000e-04-0.00021j -2.7570e-02+0.02119j  3.4361e-01+0.j     ]] 

[[0.4+0.j 0. +0.j 0. -0.j]
 [0. -0.j 0.3+0.j 0. +0.j]
 [0. +0.j 0. -0.j 0.3+0.j]] 

[[ 0.7+0.j -0. +0.j  0. -0.j]
 [-0. -0.j  0.2+0.j -0. -0.j]
 [ 0. +0.j -0. +0.j  0.1+0.j]]
